#### Import required libraries

In [1]:
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
#from scalers import *
import pickle
import random

In [2]:
client_id = "f68226fa347346169cb5cdec05f72a8b"
client_secret ="e082fbec5984468f946e9e056e9083ce"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#### Read csv into notebook

In [3]:
song_clusters_df = pd.read_csv("../Notebooks/Data/songs_with_clusters.csv")

In [4]:
song_clusters_df.head(203)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist,hot_100,cluster
0,0.707,0.681,0,-4.325,1,0.0668,0.06320,0.000005,0.0322,0.646,...,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455,4,flowers,miley cyrus,1,2
1,0.347,0.378,8,-9.960,1,0.0354,0.85700,0.000000,0.1180,0.610,...,5ReIs3aH4wfBjp93QOtWAl,spotify:track:5ReIs3aH4wfBjp93QOtWAl,https://api.spotify.com/v1/tracks/5ReIs3aH4wfB...,https://api.spotify.com/v1/audio-analysis/5ReI...,168609,4,kill bill,sza,1,1
2,0.517,0.675,6,-5.382,1,0.0357,0.45900,0.000000,0.1510,0.518,...,59uQI0PADDKeE6UZDTJEe8,spotify:track:59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://api.spotify.com/v1/audio-analysis/59uQ...,163855,4,last night,morgan wallen,1,0
3,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,...,3nqQXoyQOWXiESFLlDF1hG,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...,156943,4,unholy,sam smith & kim petras,1,7
4,0.728,0.293,1,-13.657,0,0.0562,0.47900,0.904000,0.1180,0.152,...,4CpujwKsq4DxOIMiKr12m8,spotify:track:4CpujwKsq4DxOIMiKr12m8,https://api.spotify.com/v1/tracks/4CpujwKsq4Dx...,https://api.spotify.com/v1/audio-analysis/4Cpu...,217026,4,creepin',"metro boomin, the weeknd & 21 savage",1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,0.727,0.496,10,-10.742,0,0.0433,0.12500,0.889000,0.1040,0.458,...,5Xum4tqWPwlTxTVLVeDQPA,spotify:track:5Xum4tqWPwlTxTVLVeDQPA,https://api.spotify.com/v1/tracks/5Xum4tqWPwlT...,https://api.spotify.com/v1/audio-analysis/5Xum...,334173,4,our love,caribou,0,5
199,0.857,0.831,2,-4.664,1,0.0995,0.00158,0.000000,0.1540,0.259,...,4VqLm4L1iAy8UwwsWrZLUA,spotify:track:4VqLm4L1iAy8UwwsWrZLUA,https://api.spotify.com/v1/tracks/4VqLm4L1iAy8...,https://api.spotify.com/v1/audio-analysis/4VqL...,134062,4,i ain't trippin off nothin,ezale,0,7
200,0.699,0.388,6,-10.722,1,0.0531,0.58800,0.000002,0.0658,0.566,...,0b5w1gamS9f239Sms9guAB,spotify:track:0b5w1gamS9f239Sms9guAB,https://api.spotify.com/v1/tracks/0b5w1gamS9f2...,https://api.spotify.com/v1/audio-analysis/0b5w...,274667,4,they don't know - original mix,disciples,0,1
201,0.596,0.639,0,-6.546,1,0.0300,0.78300,0.569000,0.3310,0.233,...,0jHZ5yXOZNKvtwPGSgL0gX,spotify:track:0jHZ5yXOZNKvtwPGSgL0gX,https://api.spotify.com/v1/tracks/0jHZ5yXOZNKv...,https://api.spotify.com/v1/audio-analysis/0jHZ...,196627,4,sabali,amadou & mariam,0,1


In [5]:
song_clusters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2489 entries, 0 to 2488
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      2489 non-null   float64
 1   energy            2489 non-null   float64
 2   key               2489 non-null   int64  
 3   loudness          2489 non-null   float64
 4   mode              2489 non-null   int64  
 5   speechiness       2489 non-null   float64
 6   acousticness      2489 non-null   float64
 7   instrumentalness  2489 non-null   float64
 8   liveness          2489 non-null   float64
 9   valence           2489 non-null   float64
 10  tempo             2489 non-null   float64
 11  type              2489 non-null   object 
 12  id                2489 non-null   object 
 13  uri               2489 non-null   object 
 14  track_href        2489 non-null   object 
 15  analysis_url      2489 non-null   object 
 16  duration_ms       2489 non-null   int64  


#### Drop duplicates from previously merged dataframe

In [6]:
song_clusters_df = song_clusters_df.drop_duplicates()

#### Function to return song id

In [7]:
def search_song(title, artist, limit=1):
    try:
        result = sp.search(q=title + artist, limit=limit)
        return result["tracks"]["items"][0]["id"]
    except:
        return None

In [8]:
a = search_song("asdsadasd", "sdafsfdgf")

#### Function that receives input of song title and artist from user

In [9]:
def read_song_and_artist():
    title_s = input("Please Enter the song title: ").lower()
    artist_s = input("Please Enter the artist title: ").lower()
    
    return artist_s, title_s

In [10]:
artist_s, title_s = read_song_and_artist()
input_s = search_song(title_s, artist_s, 1)

Please Enter the song title:  mockingbird
Please Enter the artist title:  eminem


In [11]:
my_dict = sp.audio_features(input_s)
my_dict

[{'danceability': 0.637,
  'energy': 0.678,
  'key': 0,
  'loudness': -3.798,
  'mode': 1,
  'speechiness': 0.266,
  'acousticness': 0.209,
  'instrumentalness': 0,
  'liveness': 0.156,
  'valence': 0.254,
  'tempo': 84.039,
  'type': 'audio_features',
  'id': '561jH07mF1jHuk7KlaeF0s',
  'uri': 'spotify:track:561jH07mF1jHuk7KlaeF0s',
  'track_href': 'https://api.spotify.com/v1/tracks/561jH07mF1jHuk7KlaeF0s',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/561jH07mF1jHuk7KlaeF0s',
  'duration_ms': 250760,
  'time_signature': 4}]

In [12]:
input_df = pd.DataFrame(my_dict)

In [13]:
input_s = input_df.drop(['type','id','uri','track_href','analysis_url',
                                    'time_signature','key','mode'],axis=1)
input_s

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.637,0.678,-3.798,0.266,0.209,0,0.156,0.254,84.039,250760


#### Function to load files

In [14]:
def load(filename = "../scalers/scaler.pickle"): 
    try: 
        with open(filename, "rb") as file: 
            return pickle.load(file) 
    except FileNotFoundError: 
        print("File ", filename, " not found!")
        return None

In [15]:
scaler = load("../Notebooks/Scaler/scaler.pickle")
scaler

StandardScaler()

In [16]:
best_model = load("../Notebooks/inertia/inertia8.pickle")

In [17]:
song_scaled = scaler.transform(input_s)

In [18]:
song_c = best_model.predict(song_scaled)

/Users/aloaskari/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [19]:
song_c

array([6], dtype=int32)

In [20]:
cluster_n = song_clusters_df[song_clusters_df['cluster'] == song_c[0]] 
recommend = (cluster_n.iloc[:,18:20]).sample(5)

In [21]:
recommend

,title,artist
156,check,young thug
1423,deja vu,prince royce
1203,let me try,kaleidoscope
1310,3 wheel-ups (feat. wiley & giggs),kano
1035,0 to 100 / the catch up,drake


#### Final Loop 

In [26]:
done = False

#Load the scaler, model and dataframe to be used

scaler = load("../Notebooks/Scaler/scaler.pickle")
model  = load("../Notebooks/inertia/inertia8.pickle")
songs  = song_clusters_df


while not done:
    
    #Let the user input a song and artist using the function
    title,artist = read_song_and_artist()
    
    #Get the id of the song input
    id = search_song(title, artist)
    if not id:
        print("! Unable to get song id")
        continue
        
    #Get the audio features of the song input
    af = sp.audio_features(id)
    if not af:
        print("! Unable to get audio features")
        continue #Restarts the while loop if there are no song features
    
    #Use the model to predict the cluster of the new song
    af_scaled_df = pd.DataFrame(af)
    af_scaled_df.drop(['type','id','uri','track_href','analysis_url',
                                    'time_signature','key','mode'], axis=1, inplace=True)
    af_scaled = scaler.transform(af_scaled_df)
    af_scaled_df = pd.DataFrame(af_scaled, columns=af_scaled_df.columns)
    
    cluster = model.predict(af_scaled_df)[0]
    
    #Check if the song is in the songs file and if it is a hot_100 song then recommend 5 hot_100 songs in the same cluster. 
    if not songs[(songs["id"] == id) & (songs["hot_100"] == 1)].empty:
        recommended = songs[(songs["hot_100"] == 1) & (songs["cluster"] == cluster)].sample(5)
    #If not recommend 5 songs from the not_100 list and the same cluster.
    else:
        recommended = songs[(songs["hot_100"] == 0) & (songs["cluster"] == cluster)].sample(5)
    
    #Return the recommended artists and song titles
    artist = recommended["artist"].values[0:5]
    title  = recommended["title"].values[0:5]
    print("Try these songs:")
    for i in range(5):
        print(" {} - {}".format(artist[i], title[i]))
    
    #Option to exit loop if user doesn't want anymore recommendations. 
    quit = input("Wanna quit? (y|n) ")
    if quit == "y":
        done = True
        print("Good bye")

Please Enter the song title:  shutdown
Please Enter the artist title:  skepta


! Unable to get song id


Please Enter the song title:  imma ride
Please Enter the artist title:  young thug


! Unable to get song id


KeyboardInterrupt: Interrupted by user